---
title: Redfield Issue check "Analytically"
date: 2024-08-08
authors:
  - name: Gerardo Suarez
---

Since there seems to be an issue with the Bloch-Redfield Solver in qutip and my not so good implementation 
of the time dependent redfield, here's a quick sympy check to make sure it's not the solvers

In [1]:
from sympy import *
from sympy.physics.quantum import Dagger
from sympy import I
from collections import defaultdict
import numpy as np
import itertools
from sympy.solvers.ode.systems import dsolve_system

In [2]:
a = symbols('a',real=True,positive=True)

In [3]:
H = diag(-a,a,a,-a)#the times 20 is to rescale time
H

Matrix([
[-a, 0, 0,  0],
[ 0, a, 0,  0],
[ 0, 0, a,  0],
[ 0, 0, 0, -a]])

In [4]:
b= symbols('b0:8', real=True)

Here I define the coupling operator that make things break for Bloch-Redfield on the SYK model

In [5]:
# coupling operator non trivial like in the shinsey ryu paper. This difference is not dramatic for simpler couplings
Q = Matrix([[0, b[0] - I*b[1], b[2] - I*b[3], 0],
              [b[0] + I*b[1], 0, 0, -b[2] + I*b[3]],
              [b[2] + I*b[3], 0, 0, b[0] - I*b[1]],
              [0, -b[2] - I*b[3], b[0] + I*b[1], 0]])
Q

Matrix([
[        0,  b0 - I*b1, b2 - I*b3,          0],
[b0 + I*b1,          0,         0, -b2 + I*b3],
[b2 + I*b3,          0,         0,  b0 - I*b1],
[        0, -b2 - I*b3, b0 + I*b1,          0]])

In [6]:
Dagger(Q) == Q #checking I didn't mess up 

True

In [7]:
def comm(a,b):
    return (a*b-b*a)

I then get the eigenvalues and eigenvectors

In [8]:
basis=H.eigenvects() # I probably don't need this step since the Hamiltonian is
# Diagonal
energies=[[i[0]]*i[1] for i in basis]
energies = [x for xs in energies for x in xs]
states = [i[2] for i in basis]
states = [x for xs in states for x in xs ]

In [9]:
def jump_operators(H,evals,all_state, Q):
    """
    A function to obtain the jump operators
    """
    collapse_list = []
    ws = []
    N=len(energies)
    for j in range(N):
        for k in range(j + 1, N):
            Deltajk = evals[k] - evals[j]
            ws.append(Deltajk)
            collapse_list.append(
                (
                    all_state[j]
                    * Dagger(all_state[j])
                    * Q
                    * all_state[k]
                    * Dagger(all_state[k])
                )
            )  # emission
            ws.append(-Deltajk)
            collapse_list.append(
                (
                    all_state[k]
                    * Dagger(all_state[k])
                    * Q
                    * all_state[j]
                    * Dagger(all_state[j])
                )
            )  # absorption
    collapse_list.append(Q - sum(collapse_list,zeros(N)))  # Dephasing
    ws.append(0)
    output = defaultdict(list)
    for k, key in enumerate(ws):
        output[key].append(collapse_list[k])
    eldict = {x: sum(y, zeros(N)) for x, y in output.items()}
    dictrem = {}
    empty = 0*H
    for keys, values in eldict.items():
        if not (values == empty):
            dictrem[keys] = values
    return dictrem

In [10]:
jumps=jump_operators(H,energies,states,Q)

### Jump operator checks

The jump operators must satisfy

$$[H,A(\omega)]=-\omega A(\omega)$$
$$[H,A^{\dagger}(\omega) A(\omega)]=0$$
$$\sum_{w} A(\omega)= A$$

In [11]:
comm(H,jumps[2*a])/(-2*a) == jumps[2*a] #checking jump decomposition is fine

True

In [12]:
comm(H,Dagger(jumps[2*a])*jumps[2*a]) == zeros(4)

True

In [13]:
jumps[2*a]+jumps[-2*a] ==Q

True

### Constucting the Differential equations
We now construct the differential equations from the GKLS form of the bloch 
Redfield generator

$$\begin{aligned}
     \dot{\rho_{S}^{I}(t)}{t} &=
    \sum_{\omega,\omega',\alpha,\beta} \gamma_{\beta,\alpha}(\omega,\omega')  \left( S_{\alpha}(\omega')  \rho^{I}_{S}(t) S^{\dagger}_{\beta}(\omega)- \frac{ \{S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega'),\rho^{I}_{S}(t)\}}{2} \right) \nonumber \\
    &+i \sum_{\omega,\omega',\alpha,\beta} S_{\beta,\alpha}(\omega,\omega') \Big[ \rho^{I}_{S}(t), S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega')\Big]\end{aligned}$$

I solve in the interaction picture generally, I did the same in my numerics so it should not be an issue (I rotate in the end).
By neglecting Lambshift as in the numerics

$$
     \dot{\rho_{S}^{I}(t)}{t} =
    \sum_{\omega,\omega',\alpha,\beta} \gamma_{\beta,\alpha}(\omega,\omega')  \left( S_{\alpha}(\omega')  \rho^{I}_{S}(t) S^{\dagger}_{\beta}(\omega)- \frac{ \{S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega'),\rho^{I}_{S}(t)\}}{2} \right)$$


In [14]:
# Simply define the symbolic density operator
rho = symbols('rho_1:17')
rho = Matrix(
    [rho[0: 4],
     rho[4: 8],
     rho[8: 12],
     rho[12:]]).subs(
    rho[-1],
    1 - rho[0] - rho[5] - rho[10])
rho=rho.subs({rho[4]: conjugate(rho[1]), rho[8]: conjugate(rho[2]),
          rho[12]:conjugate(rho[3]),rho[6]:conjugate(rho[9]),
          rho[13]: conjugate(rho[7]), rho[14]:conjugate(rho[11])})
rho


Matrix([
[           rho_1,            rho_2,             rho_3,                       rho_4],
[conjugate(rho_2),            rho_6, conjugate(rho_10),                       rho_8],
[conjugate(rho_3),           rho_10,            rho_11,                      rho_12],
[conjugate(rho_4), conjugate(rho_8), conjugate(rho_12), -rho_1 - rho_11 - rho_6 + 1]])

In [15]:
# This bit is simply the GKLS form without the appropiate coefficients
def matrix_form(jumps, combinations,rho):
    matrixform = {}
    for i in combinations:
        matrixform[i] = (
            jumps[i[1]]*rho *Dagger(jumps[i[0]]) -
            (0.5 *
                (Dagger(jumps[i[0]]) * jumps[i[1]]*rho +
                rho*Dagger(jumps[i[0]])* jumps[i[1]])))
    return matrixform

As the sum goes on $(\omega,\omega')$ pairs I construct all combinations

In [16]:
ws = list(jumps.keys())
combinations = list(itertools.product(ws, ws))

I get the GKLS form of each of those combinations, as a dictionary

In [17]:
matrices=matrix_form(jumps,combinations,rho)

Then I construct the generator by multiplying the appropiate coefficient to each of the GKLS from matrices. Now for the coefficients we have

$$\begin{aligned}
    \Gamma_{\alpha,\beta}(\omega,t) = \int_{0}^{t} ds e^{i \omega s}  \langle B^{\dagger}_{\alpha}(t) B_{\beta}^{\dagger}(t-s) \rangle_{B} \end{aligned}$$

For convenience we also define

$$\begin{aligned}
    \Gamma_{\alpha,\beta}(\omega,\omega',t) = e^{i (\omega'-\omega) t} \int_{0}^{t} ds e^{i \omega s}  \langle B_{\alpha}^{\dagger}(t) B_{\beta}(t-s) \rangle_{B} =e^{i (\omega'-\omega) t} \Gamma_{\alpha,\beta}(\omega,t) \end{aligned}$$

Since I mainly care about bloch-redfield I make $t \xrightarrow{} \infty$ (in the integral) so

$$\begin{aligned}
    \Gamma_{\alpha,\beta}(\omega,\omega',t) = e^{i (\omega'-\omega) t} \int_{0}^{\infty} ds e^{i \omega s}  \langle B(s) B(0) \rangle_{B} =e^{i (\omega'-\omega) t} \Gamma_{\alpha,\beta}(\omega) \end{aligned}$$

Where $\Gamma_{\alpha,\beta}(\omega)$  is the power spectrum

In [18]:
lam,T,w0,t,gam=symbols("lambda T w0 t gamma",real=True)

In [19]:
def power_spectrum(w):
    if w==0:
        return 2 * lam**2 * gam / ( w0**4 * 1/T)
    else:
        return 2 * lam**2 * gam * w / ((w0**2 - w**2)**2 + gam**2 * w**2) * ((1 / (exp(w /T) - 1)) + 1)

In [20]:
def generator(combinations, matrices):
    gen = [
        limit(exp(I*(s[1]-s[0])* t)*power_spectrum(s[0]),T,0) * matrices[s]
        for s in combinations]
    return sum(gen, zeros(4))      

In [21]:
gene = generator(combinations,matrices)

In [22]:
gene

Matrix([
[                                                                                                                                                                                                                                                  4*a*gamma*lambda**2*((b0 + I*b1)*(rho_10*(b2 - I*b3) + rho_6*(b0 - I*b1)) + (b2 + I*b3)*(rho_11*(b2 - I*b3) + (b0 - I*b1)*conjugate(rho_10)))/(4*a**2*gamma**2 + (-4*a**2 + w0**2)**2),                                                                                                                                                             4*a*gamma*lambda**2*(-0.5*(b0 - I*b1)*(rho_2*(b0 + I*b1) + rho_3*(b2 + I*b3)) - 0.5*(-b2 - I*b3)*(rho_2*(-b2 + I*b3) + rho_3*(b0 - I*b1)))/(4*a**2*gamma**2 + (-4*a**2 + w0**2)**2),                                                                                                                                                                         4*a*gamma*lambda**2*(-0.5*(b0 + I*b1)*(rho_2*(-b2 + I*b3) +

Next we simply vectorize the density matrix

In [23]:
mgene = Matrix(list(gene))
rhom=Matrix(list(rho))

In [24]:
rhof = symbols('rho_1:17',cls=Function) # This is necessary to solve the differential equation
to_funcs={i:rhof[k](t) for k,i in enumerate(rhom)} #could have done so from the beginning though

In [25]:
mgene=mgene.subs(to_funcs) #substitute symbols for a function
mrhof=rhom.subs(to_funcs)

At this stage we have the system of differential equations. We need to solve it

In [42]:
eqs = [Eq(mrhof[k].diff(t), mgene[k].simplify()) for k in range(16)]

In [27]:
for i in eqs:
    display(i)

Eq(Derivative(rho_1(t), t), 4*a*gamma*lambda**2*((b0 + I*b1)*((b0 - I*b1)*rho_6(t) + (b2 - I*b3)*rho_10(t)) + (b2 + I*b3)*((b0 - I*b1)*rho_7(t) + (b2 - I*b3)*rho_11(t)))/(4*a**2*gamma**2 + (4*a**2 - w0**2)**2))

Eq(Derivative(rho_2(t), t), 2.0*a*gamma*lambda**2*(-b0**2 - b1**2 - b2**2 - b3**2)*rho_2(t)/(16.0*a**4 + 4.0*a**2*gamma**2 - 8.0*a**2*w0**2 + 1.0*w0**4))

Eq(Derivative(rho_3(t), t), 2.0*a*gamma*lambda**2*(-b0**2 - b1**2 - b2**2 - b3**2)*rho_3(t)/(16.0*a**4 + 4.0*a**2*gamma**2 - 8.0*a**2*w0**2 + 1.0*w0**4))

Eq(Derivative(rho_4(t), t), 4*a*gamma*lambda**2*((b0 - I*b1)*((b0 - I*b1)*rho_7(t) + (b2 - I*b3)*rho_11(t)) - (b2 - I*b3)*((b0 - I*b1)*rho_6(t) + (b2 - I*b3)*rho_10(t)))/(4*a**2*gamma**2 + (4*a**2 - w0**2)**2))

Eq(Derivative(rho_5(t), t), a*gamma*lambda**2*(4*(b0 + I*b1)*((b0 + I*b1)*rho_2(t) - (b2 - I*b3)*rho_14(t)) + 4*(b2 + I*b3)*((b0 + I*b1)*rho_3(t) - (b2 - I*b3)*rho_15(t)) - 2.0*((b0 - I*b1)*(b0 + I*b1) + (b2 - I*b3)*(b2 + I*b3))*rho_5(t)*exp(4*I*a*t))*exp(-4*I*a*t)/(4*a**2*gamma**2 + (4*a**2 - w0**2)**2))

...

Eq(Derivative(rho_12(t), t), a*gamma*lambda**2*(4*(b0 - I*b1)*((b0 - I*b1)*rho_15(t) + (b2 + I*b3)*rho_3(t)) - 4*(b2 - I*b3)*((b0 - I*b1)*rho_14(t) + (b2 + I*b3)*rho_2(t)) - 2.0*((b0 - I*b1)*(b0 + I*b1) + (b2 - I*b3)*(b2 + I*b3))*rho_12(t)*exp(4*I*a*t))*exp(-4*I*a*t)/(4*a**2*gamma**2 + (4*a**2 - w0**2)**2))

Eq(Derivative(rho_13(t), t), 4*a*gamma*lambda**2*((b0 + I*b1)*((b0 + I*b1)*rho_10(t) - (b2 + I*b3)*rho_6(t)) + (b2 + I*b3)*((b0 + I*b1)*rho_11(t) - (b2 + I*b3)*rho_7(t)))/(4*a**2*gamma**2 + (4*a**2 - w0**2)**2))

Eq(Derivative(rho_14(t), t), 2.0*a*gamma*lambda**2*(-b0**2 - b1**2 - b2**2 - b3**2)*rho_14(t)/(16.0*a**4 + 4.0*a**2*gamma**2 - 8.0*a**2*w0**2 + 1.0*w0**4))

Eq(Derivative(rho_15(t), t), 2.0*a*gamma*lambda**2*(-b0**2 - b1**2 - b2**2 - b3**2)*rho_15(t)/(16.0*a**4 + 4.0*a**2*gamma**2 - 8.0*a**2*w0**2 + 1.0*w0**4))

Eq(Derivative(rho_16(t), t), 4*a*gamma*lambda**2*((b0 - I*b1)*((b0 + I*b1)*rho_11(t) - (b2 + I*b3)*rho_7(t)) - (b2 - I*b3)*((b0 + I*b1)*rho_10(t) - (b2 + I*b3)*rho_6(t)))/(4*a**2*gamma**2 + (4*a**2 - w0**2)**2))

Let me make a few change of variables, anc call the new variables $c_{k}$

In [38]:
c=symbols("c0:10")

In [49]:
c0=mgene[14].simplify()/mrhof[14]
c0

2.0*a*gamma*lambda**2*(-b0**2 - b1**2 - b2**2 - b3**2)/(16.0*a**4 + 4.0*a**2*gamma**2 - 8.0*a**2*w0**2 + 1.0*w0**4)

In [59]:
c1=(gam*lam**2)/fraction(mgene[0])[1].simplify()
c1

gamma*lambda**2/(4*a**2*gamma**2 + (4*a**2 - w0**2)**2)

In [60]:
for i in eqs:
    display(i.subs(c0,c[0]).subs(2*c0,2*c[0]).subs(c1,c[1]))

Eq(Derivative(rho_1(t), t), 4*a*c1*((b0 + I*b1)*((b0 - I*b1)*rho_6(t) + (b2 - I*b3)*rho_10(t)) + (b2 + I*b3)*((b0 - I*b1)*rho_7(t) + (b2 - I*b3)*rho_11(t))))

Eq(Derivative(rho_2(t), t), c0*rho_2(t))

Eq(Derivative(rho_3(t), t), c0*rho_3(t))

Eq(Derivative(rho_4(t), t), 4*a*c1*((b0 - I*b1)*((b0 - I*b1)*rho_7(t) + (b2 - I*b3)*rho_11(t)) - (b2 - I*b3)*((b0 - I*b1)*rho_6(t) + (b2 - I*b3)*rho_10(t))))

Eq(Derivative(rho_5(t), t), a*c1*(4*(b0 + I*b1)*((b0 + I*b1)*rho_2(t) - (b2 - I*b3)*rho_14(t)) + 4*(b2 + I*b3)*((b0 + I*b1)*rho_3(t) - (b2 - I*b3)*rho_15(t)) - 2.0*((b0 - I*b1)*(b0 + I*b1) + (b2 - I*b3)*(b2 + I*b3))*rho_5(t)*exp(4*I*a*t))*exp(-4*I*a*t))

...

Eq(Derivative(rho_12(t), t), a*c1*(4*(b0 - I*b1)*((b0 - I*b1)*rho_15(t) + (b2 + I*b3)*rho_3(t)) - 4*(b2 - I*b3)*((b0 - I*b1)*rho_14(t) + (b2 + I*b3)*rho_2(t)) - 2.0*((b0 - I*b1)*(b0 + I*b1) + (b2 - I*b3)*(b2 + I*b3))*rho_12(t)*exp(4*I*a*t))*exp(-4*I*a*t))

Eq(Derivative(rho_13(t), t), 4*a*c1*((b0 + I*b1)*((b0 + I*b1)*rho_10(t) - (b2 + I*b3)*rho_6(t)) + (b2 + I*b3)*((b0 + I*b1)*rho_11(t) - (b2 + I*b3)*rho_7(t))))

Eq(Derivative(rho_14(t), t), c0*rho_14(t))

Eq(Derivative(rho_15(t), t), c0*rho_15(t))

Eq(Derivative(rho_16(t), t), 4*a*c1*((b0 - I*b1)*((b0 + I*b1)*rho_11(t) - (b2 + I*b3)*rho_7(t)) - (b2 - I*b3)*((b0 + I*b1)*rho_10(t) - (b2 + I*b3)*rho_6(t))))

In [28]:
# sols = dsolve_system(
#     eqs,
#     ics={rhof[0] (0): 0, rhof[1] (0): 0, rhof[2] (0): 0, rhof[3] (0): 0,
#          rhof[4] (0): 0, rhof[5] (0): 0.5, rhof[6] (0): 0.5 * I, rhof[7]
#           (0): 0, rhof[8] (0): 0, rhof[9] (0): -0.5 * I, rhof[10] (0): 0.5,
#          rhof[11](0): 0, rhof[12](0): 0, rhof[13](0): 0, rhof[14](0): 0,
#          rhof[15](0): 0})

Solving Fully symbolically is too slow. I need to try some ratios to have a simpler set of symbols so it does not run forever. One can set numerical values (In a sense this is not much different than the solvers,
On the other hand the system of equations is till solved analytically, no quadratures are used)

:::{warning} 
Some of the equations are actually pretty easy to solve symbolically for example
$\rho_{14},\rho_{15},\rho_{11},\rho_{10},\rho_{6},\rho_{7},\rho_{3},\rho_{2}$ all have really simple solutions. Eventhough sympy is having a hard time. I think it might be worth reparametrizing and solving here or simply solving by hand. In the next few days
:::


In [29]:
num_values = {w0: 4.799892154530513, lam: 2.157529680721121, gam: 2.3414108070880553, a: 2.182,b[0]:0.53,b[1]:0.707,b[2]:0.442,b[3]:0.471}

In [30]:
eqs = [Eq(mrhof[k].diff(t), mgene[k].subs(num_values).simplify()) for k in range(16)]

In [31]:
eqs[0]

Eq(Derivative(rho_1(t), t), 1.0*(0.448330014085309 + 0.0496843899774862*I)*rho_10(t) + 0.329736827445869*rho_11(t) + 0.617062830722391*rho_6(t) + 1.0*(0.448330014085309 - 0.0496843899774862*I)*rho_7(t))

In [32]:
for i in eqs:
    display(i)

Eq(Derivative(rho_1(t), t), 1.0*(0.448330014085309 + 0.0496843899774862*I)*rho_10(t) + 0.329736827445869*rho_11(t) + 0.617062830722391*rho_6(t) + 1.0*(0.448330014085309 - 0.0496843899774862*I)*rho_7(t))

Eq(Derivative(rho_2(t), t), -0.47339982908413*rho_2(t))

Eq(Derivative(rho_3(t), t), -0.47339982908413*rho_3(t))

Eq(Derivative(rho_4(t), t), 1.0*(0.0209260243292489 + 0.329072145402553*I)*rho_10(t) - 1.0*(0.0780365171355156 + 0.444273161614031*I)*rho_11(t) + 1.0*(0.0780365171355156 + 0.444273161614031*I)*rho_6(t) - 1.0*(0.173045741619697 + 0.592302041501141*I)*rho_7(t))

Eq(Derivative(rho_5(t), t), 1.0*(-(0.448330014085309 + 0.0496843899774862*I)*rho_14(t) - 0.329736827445869*rho_15(t) + (-0.173045741619697 + 0.592302041501141*I)*rho_2(t) + (-0.0780365171355156 + 0.444273161614031*I)*rho_3(t) - 0.47339982908413*rho_5(t)*exp(8.728*I*t))*exp(-8.728*I*t))

...

Eq(Derivative(rho_12(t), t), 1.0*(-0.47339982908413*rho_12(t)*exp(8.728*I*t) + (0.0780365171355156 + 0.444273161614031*I)*rho_14(t) - (0.173045741619697 + 0.592302041501141*I)*rho_15(t) - 0.329736827445869*rho_2(t) + (0.448330014085309 - 0.0496843899774862*I)*rho_3(t))*exp(-8.728*I*t))

Eq(Derivative(rho_13(t), t), 1.0*(-0.173045741619697 + 0.592302041501141*I)*rho_10(t) + 1.0*(-0.0780365171355156 + 0.444273161614031*I)*rho_11(t) + 1.0*(0.0780365171355156 - 0.444273161614031*I)*rho_6(t) + 1.0*(0.0209260243292489 - 0.329072145402553*I)*rho_7(t))

Eq(Derivative(rho_14(t), t), -0.47339982908413*rho_14(t))

Eq(Derivative(rho_15(t), t), -0.47339982908413*rho_15(t))

Eq(Derivative(rho_16(t), t), -1.0*(0.448330014085309 + 0.0496843899774862*I)*rho_10(t) + 0.617062830722391*rho_11(t) + 0.329736827445869*rho_6(t) + 1.0*(-0.448330014085309 + 0.0496843899774862*I)*rho_7(t))

In [33]:
sols = dsolve_system(
    eqs,
    ics={rhof[0] (0): 0, rhof[1] (0): 0, rhof[2] (0): 0, rhof[3] (0): 0,
         rhof[4] (0): 0, rhof[5] (0): 0.5, rhof[6] (0): 0.5 * I, rhof[7]
          (0): 0, rhof[8] (0): 0, rhof[9] (0): -0.5 * I, rhof[10] (0): 0.5,
         rhof[11](0): 0, rhof[12](0): 0, rhof[13](0): 0, rhof[14](0): 0,
         rhof[15](0): 0})

KeyboardInterrupt: 

In [ ]:
ans=Matrix(np.array([i.rhs for i in sols[0]]).reshape(4,4))

In [ ]:
def roundMatrix(m, n): 
    return Matrix(
    [[round(m[x, y], n) for y in range(m.shape[1])] for x in range(m.shape[0])])

In [ ]:
roundMatrix(ans.subs(t,50),10)

I believe I was careful enough to use the same convention used in the other equations (Pseudomodes, Cumulant and redfield ) but currently reviewing the derivations to make sure there's no inconsistencies. The derivations in question are in https://master--gsuarezthesis.netlify.app/redfield . I do think it is now safe to assume that BR/Redfield breaksdown and that it is not a bug in code (Unless the three different implementations have the same bug xd), so maybe we can write a paper on redfield breaking down, cumulant/global being good once we figure out why it happens🤓

:::{tip} About Pictures
Technically the above matrix is not correct as it is in the interaction picture and not the Schrodinger picture. In this case it does not make a difference, however, let us do the rotation 
:::

In [ ]:
U=exp(I*H*t)
U

In [ ]:
U*rho*Dagger(U) ## To confirm there are no differences in the steady state see that the observables
## that remain in the steady state are the same in both pictures

In [ ]:
U*roundMatrix(ans.subs(t,50),10)*Dagger(U)

## RC picture of the Hamiltonian

For the RC I simply follow one of your papers https://arxiv.org/pdf/1511.05181

So If I didn't misunderstand it then

$$\Omega=w_{0}$$
$$\lambda_{rc}=\sqrt{\frac{\pi}{2 w_{0}}}\lambda$$


Not actually sure if $\pi$ should be there, but does not seem to be relevant for the question we are asking

In [ ]:
H

In [ ]:
def ladder(i,j):
    if i - j == 1:
        return sqrt(i)
    else:
        return 0
def destroy(n):
    return Dagger(Matrix(n,n,ladder))


Not sure how many levels to take here, but let us guess 15 is enough

In [ ]:
levels=15
arc=destroy(levels)
arcd=Dagger(arc)

Then I construct the RC Hamiltonian

In [ ]:
from sympy.physics.quantum import TensorProduct

In [ ]:
HRC = TensorProduct(H, eye(levels))+sqrt(pi/(2*w0))*lam*TensorProduct(Q,eye(levels))*(TensorProduct(eye(4),arc)+TensorProduct(eye(4),arcd))+w0*TensorProduct(eye(4),arcd)*TensorProduct(eye(4),arc)

In [ ]:
NHRC=HRC.subs(num_values).evalf()

In [ ]:
ans=np.array(NHRC.tolist()).astype(np.complex128)

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(ans)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(np.real(eigenvalues),np.round(np.imag(eigenvalues),10),s=5)
plt.title("Eigenvalues on the real line")
plt.show()

Propably I shopuld have done the partial trace so

In [ ]:
import qutip as qt

In [ ]:
Hq=qt.Qobj(H.subs(num_values).tolist())
aq=qt.destroy(levels)
aqd=qt.create(levels)
qHRC = qt.tensor(Hq, qt.qeye(levels))+np.sqrt(np.pi/(2*num_values[w0]))*num_values[lam]*qt.tensor(qt.Qobj(Q.subs(num_values)), qt.qeye(levels))*(qt.tensor(qt.qeye(4),aq)+qt.tensor(qt.qeye(4),aqd))+num_values[w0]*qt.tensor(qt.qeye(4),aqd)*qt.tensor(qt.qeye(4),aq)

In [ ]:
Hq

In [ ]:
qHRC.ptrace(0)

Still degenerate

Let us try the other coupling which does not break bloch redfield

In [ ]:
Hq=qt.Qobj(H.subs(num_values).tolist())
aq=qt.destroy(levels)
aqd=qt.create(levels)
qHRC = qt.tensor(Hq, qt.qeye(levels))+np.sqrt(np.pi/(2*num_values[w0]))*num_values[lam]*qt.tensor(qt.Qobj(re(Q).subs(num_values)), qt.qeye(levels))*(qt.tensor(qt.qeye(4),aq)+qt.tensor(qt.qeye(4),aqd))+num_values[w0]*qt.tensor(qt.qeye(4),aqd)*qt.tensor(qt.qeye(4),aq)

In [ ]:
qHRC.ptrace(0)

In [ ]:
ans = qHRC.full().astype(np.complex128)

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(ans)

In [ ]:
plt.scatter(np.real(eigenvalues), np.round(np.imag(eigenvalues), 10), s=5)
plt.title("Eigenvalues on the real line")
plt.show()

There doesn't seem to be much change in the Hamiltonian if any